Source:

https://answers.opencv.org/question/194679/detecting-embossed-texts-characterstechniques/

https://pyimagesearch.com/2017/07/17/credit-card-ocr-with-opencv-and-python/

In [ ]:
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2

Load the reference OCR-A image from disk, convert it to grayscale, and threshold it, such that the digits appear as *white* on a *black* background and invert it, such that the digits appear as *white* on a *black*

In [ ]:
ref = cv2.imread("./in/ocr-b-font.png")
ref = cv2.cvtColor(ref, cv2.COLOR_BGR2GRAY)
ref = cv2.threshold(ref, 10, 255, cv2.THRESH_BINARY_INV)[1]

Find contours in the OCR-A image (i.e,. the outlines of the digits) sort them from left to right, and initialize a dictionary to map digit name to the ROI

In [ ]:
refCnts = cv2.findContours(ref.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
refCnts = imutils.grab_contours(refCnts)
refCnts = contours.sort_contours(refCnts, method="left-to-right")[0]
characters = {}

At this point, we should loop through the contours, extract, and associate ROIs with their corresponding characters:

In [ ]:
for (i, c) in enumerate(refCnts):
	# compute the bounding box for the digit, extract it, and resize
	# it to a fixed size
	(x, y, w, h) = cv2.boundingRect(c)
	roi = ref[y:y + h, x:x + w]
	roi = cv2.resize(roi, (57, 88))
	# update the characters dictionary, mapping the digit name to the ROI
	characters[i] = roi

At this point, we are done extracting the digits from our reference image and associating them with their corresponding digit name.

Let’s continue by initializing a couple structuring kernels:

In [ ]:
# initialize a rectangular (wider than it is tall) and square
# structuring kernel
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3))
sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

Now let’s prepare the image we are going to OCR:

In [ ]:
image = cv2.imread("./in/card_input.jpg")
image = imutils.resize(image, width=300)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imwrite("./out/embrossed_text/gray.jpg", gray)

Now that our image is grayscaled and the size is consistent, let’s perform a morphological operation:

In [ ]:
# apply a tophat (whitehat) morphological operator to find light
# regions against a dark background (i.e., the credit card numbers)
tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, rectKernel)
cv2.imwrite("./out/embrossed_text/tophat.jpg", tophat)

Given our tophat image, let’s compute the gradient along the x-direction:

In [ ]:
# compute the Scharr gradient of the tophat image, then scale
# the rest back into the range [0, 255]
gradX = cv2.Sobel(tophat, ddepth=cv2.CV_32F, dx=1, dy=0,
	ksize=-1)
gradX = np.absolute(gradX)
(minVal, maxVal) = (np.min(gradX), np.max(gradX))
gradX = (255 * ((gradX - minVal) / (maxVal - minVal)))
gradX = gradX.astype("uint8")
cv2.imwrite("./out/embrossed_text/gradX.jpg", gradX)
# Computing the Scharr gradient magnitude representation of the image reveals vertical changes in the gradient.

Let’s continue to improve our credit card digit finding algorithm:

In [53]:
# apply a closing operation using the rectangular kernel to help
# cloes gaps in between credit card number digits, then apply
# Otsu's thresholding method to binarize the image
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel)
cv2.imwrite("./out/embrossed_text/gradX_close.jpg", gradX)
thresh = cv2.threshold(gradX, 0, 255,
	cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
cv2.imwrite("./out/embrossed_text/thresh.jpg", thresh)
# apply a second closing operation to the binary image, again
# to help close gaps between credit card number regions
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
cv2.imwrite("./out/embrossed_text/thresh_close.jpg", thresh)

Next let’s find the contours and initialize the list of character grouping locations.

In [ ]:
# find contours in the thresholded image, then initialize the
# list of digit locations
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

Now let’s loop through the contours while filtering based on the aspect ratio of each, allowing us to prune the digit group locations from other, irrelevant areas of the credit card:

In [ ]:
locs = []
# loop over the contours
for (i, c) in enumerate(cnts):
	# compute the bounding box of the contour, then use the
	# bounding box coordinates to derive the aspect ratio
	(x, y, w, h) = cv2.boundingRect(c)
	ar = w / float(h)
	# since credit cards used a fixed size fonts with 4 groups
	# of 4 digits, we can prune potential contours based on the
	# aspect ratio
	if ar > 2.5 and ar < 4.0:
		# contours can further be pruned on minimum/maximum width
		# and height
		if (w > 40 and w < 55) and (h > 10 and h < 20):
			# append the bounding box region of the digits group
			# to our locations list
			locs.append((x, y, w, h))

print("locs:", len(locs))

Next, we’ll sort the groupings from left to right and initialize a list for the credit card digits:

In [ ]:
# sort the digit locations from left-to-right, then initialize the
# list of classified digits
locs = sorted(locs, key=lambda x:x[0])
output = []

Now that we know where each group of four digits is, let’s loop through the four sorted groupings and determine the digits therein.

In [ ]:
# loop over the 4 groupings of 4 digits
for (i, (gX, gY, gW, gH)) in enumerate(locs):
	# initialize the list of group digits
	groupOutput = []
	# extract the group ROI of 4 digits from the grayscale image,
	# then apply thresholding to segment the digits from the
	# background of the credit card
	group = gray[gY - 5:gY + gH + 5, gX - 5:gX + gW + 5]
	group = cv2.threshold(group, 0, 255,
		cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
	# detect the contours of each individual digit in the group,
	# then sort the digit contours from left to right
	digitCnts = cv2.findContours(group.copy(), cv2.RETR_EXTERNAL,
		cv2.CHAIN_APPROX_SIMPLE)
	digitCnts = imutils.grab_contours(digitCnts)
	digitCnts = contours.sort_contours(digitCnts,
		method="left-to-right")[0]